# Задание 1

[Свертка списка](https://ru.wikipedia.org/wiki/%D0%A1%D0%B2%D1%91%D1%80%D1%82%D0%BA%D0%B0_%D1%81%D0%BF%D0%B8%D1%81%D0%BA%D0%B0) -  это обобщенная операция над списком, c помощью которой можно преобразовать список в единое значение. Например, рассмотрим реализации свертки слева и свертки справа (левоассоциативную свертку и правоассоциативную свертку):

In [120]:
def foldl(f, x0, lst):
    if not lst:
        return x0
    return foldl(f, f(x0, lst[0]), lst[1:])

def foldr(f, x0, lst):
    if not lst:
        return x0
    return f(lst[0], foldr(f, x0, lst[1:]))

In [121]:
#сумма
foldl(lambda x, y: x + y, 0, [1, 2, 3])

6

In [122]:
#конкатенация
foldl(lambda x, y: '{}{}'.format(x, y), '', [1, 2, 3])

'123'

In [123]:
f = lambda x, y: x / y
foldl(f, 1, [1, 2, 3]), \
foldr(f, 1, [1, 2, 3])

(0.16666666666666666, 1.5)

Задача: реализовать foldl через foldr и наоборот. Вместо многоточий нужно вставить выражения, которые бы привели к нужному результату.

In [124]:
def foldl_with_foldr(f,x0,lst):
    composed=foldr(lambda seqval,acc: lambda n: acc(f(n, seqval)),lambda identity: identity,lst)
    return composed(x0)

In [125]:
def foldr_with_foldl(f,x0,lst):
    composed=foldl(lambda acc,seqval: lambda n: acc(f(seqval,n)),lambda identity: identity,lst)
    return composed(x0)

# Задание 2

нужно написать функцию, которая принимает две строки и проверяет, входит ли хотя бы одна перестановка второй строки в первую. Например:

> a = 'abcrotm'
> 
> b = 'tro'

функция def check_inv(a, b) вернет True, так как 'rot' содержится в 'abcrotm'. Нужно подумать как можно более оптимальный алгоритм и оценить его сложность. 

In [126]:
from collections import Counter
def check_inv(a, b):
    if (len(a)<len(b)):
        return False
    a_map_letter_to_frequency=Counter(a)
    b_map_letter_to_frequency=Counter(b)
    return not bool(b_map_letter_to_frequency - a_map_letter_to_frequency)

n - length a, m - length b; \
Constructing Maps is O(n)+O(m); \
Getting from first Map is O(1) and O(1) to delete from second for one iter => 
## Complexity is: O(n+m)

# Задание 3

Реализовать бинарное дерево (класс Tree), в нём методы __repr__, __str__, __iter__ (итерация только по листьям).

In [127]:
class Tree:
    def __init__(self, value=None, left=None, right=None):
        self.left = left
        self.right = right
        self.value = value
        
    def __iter__(self):
        if self.left == None and self.right == None:
            yield self.value
        
        if self.left != None:
            yield from self.left
                
        if self.right != None:
            yield from self.right
    
    def __str__(self):
         return "BinarySearchTree(" + repr(self.root) + ")"
    
    def __repr__(self):
         return "BinarySearchTree.Node(" + repr(self.value) + "," + repr(self.left) + "," + repr(self.right) + ")"
    
tree = Tree(0, Tree(1, Tree(3), Tree(4)),Tree(2))
list(tree) == [3, 4, 2]

True

# Задание 4

Реализовать простейший калькулятор математических выражений:
- только целые числа
- **+**, **\-**, **\***, **\**
- скобки

**Можно использовать регулярные выражения**

I decided to use simplest solution of problem - Reverse Polish Notation:

In [128]:
import re
operations = {
  "+": (lambda a, b: a + b),
  "-": (lambda a, b: a - b),
  "*": (lambda a, b: a * b),
  "/": (lambda a, b: a / b)
}

Method to handle cases such -2 and (-2):

In [129]:
def hadle_special_cases(expression):
    preparedExpression=''
    for i in range(len(expression)):
        currentSymbol = expression[i]
        
        if(currentSymbol == '-'):
            if(i == 0):
                preparedExpression+='0'
                
            if(expression[i-1] == '('):
                preparedExpression+='0'
            
        preparedExpression+=currentSymbol
        
    return preparedExpression

Parse input string to RPN:

In [130]:
def parse_expression_to_RPN(expression):
    expression = hadle_special_cases(expression)
    rpnExpression=''
    operationStack=[]
    
    for i in range (len(expression)):
        currentSymbol = expression[i]
        currentPriority=getPriority(expression[i])
        
        if(re.match('[0-9]',currentSymbol)):
            rpnExpression+=currentSymbol
            
        if(currentSymbol == '('):
            operationStack.append(currentSymbol)
            
        if (re.match('[\+*/-]',currentSymbol)):
            rpnExpression+=' '
            while (operationStack):
                if (getPriority(operationStack[-1])>=currentPriority):
                    rpnExpression+=operationStack.pop()
                    rpnExpression+=' '
                else: break
            operationStack.append(currentSymbol)
                    
        if (currentPriority == -1):
            rpnExpression+=' '
            while (getPriority(operationStack[-1]) != 1):
                 rpnExpression+=operationStack.pop()
                 rpnExpression+=' '
            operationStack.pop()
       
    while (operationStack):
        rpnExpression+=' '
        rpnExpression+=operationStack.pop();
    return rpnExpression
    

parse fron RPN to digit answer:

In [131]:
def parse_RPN_to_answer(rpn):
    tokens = rpn.split()
    resultStack= []

    for token in tokens:
        if token in operations:
            b = resultStack.pop()
            a = resultStack.pop()
            result = operations[token](a, b)
            resultStack.append(result)
        else:
            resultStack.append(int(token))

    return resultStack.pop()

In [132]:
def calc(expr):
    rpn=parse_expression_to_RPN(expr)
    return parse_RPN_to_answer(rpn)

calc('2 * (15 - 3 * 4) - 2')==4

True